In [1]:
# import custom functions
from get_candidates import get_candidates
from get_row_col import get_row_col
from convert_to_binary import convert_to_binary
from naked_for_how_long import naked_for_how_long

# other functions we need
from ast import literal_eval

import pandas as pd
import numpy as np

In [2]:
# import SQL dependencies
import sqlite3

# connect to .db
con = sqlite3.connect("../data-collection/nyt-sudoku.db")
cur = con.cursor()

doks_perf = pd.read_sql("SELECT * from doks_performance;",
           con)

doks_easy = pd.read_sql("SELECT * from doks_easy;",
           con)

In [3]:
# format doks_performance date to match doks_easy format
doks_perf['date'] = pd.to_datetime(doks_perf['date'])

# format date column in doks_easy
doks_easy['date'] = pd.to_datetime(doks_easy['date'])

# convert all string of lists to lists
doks_perf['doks_order'] = doks_perf['doks_order'].apply(literal_eval)
doks_perf['times'] = doks_perf['times'].apply(literal_eval)

doks_easy['puzzle'] = doks_easy['puzzle'].apply(literal_eval)
doks_easy['solution'] = doks_easy['solution'].apply(literal_eval)

In [4]:
merged = pd.merge(left=doks_perf,
        right=doks_easy,
        on='date',
        how='left')

In [5]:
# drop those days for which we don't have original nyt data
merged = merged.dropna()

In [6]:
# only keep the columns we need
merged = merged[['date', 'puzzle', 'solution', 'doks_order', 'times']]

In [7]:
naked_num_steps = pd.DataFrame(columns=[i for i in range(81)])

In [8]:
naked_num_steps

Empty DataFrame
Columns: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80]
Index: []

[0 rows x 81 columns]

In [9]:
for record in range(len(merged)):
    temp_df = pd.DataFrame(naked_for_how_long(merged.iloc[record,3],
                  merged.iloc[record,1],
                  merged.iloc[record,2]),
                    index=[0])
    
    naked_num_steps = naked_num_steps.append(temp_df)

43
43
45
45
43
45
43
43
43
45
43
43
43
45
45
43
43
43
45
43
49
43
43
43
46
43
43
43
45
45
43
43
43
43
43
44
44
43
45
45
45
44
48
49
47
44
45
45
48
43
43
43
48
47
43
44
43
44
44
44
50
52
44
44
44
48
45
43
47
43


In [12]:
naked_num_steps.to_csv('naked_num_steps.csv', index=False)

In [15]:
naked_num_steps = naked_num_steps.replace(0, np.nan)

In [20]:
avg_steps_naked = np.array(np.round(naked_num_steps.sum(axis=0) / naked_num_steps.notna().sum(axis=0), 2)).reshape(9,9)

In [21]:
avg_num_steps_naked = np.array(np.round(naked_num_steps.sum(axis=0) / naked_num_steps.notna().sum(axis=0), 2)).mean()

In [23]:
np.round(avg_steps_naked - avg_num_steps_naked, 2)

array([[-2.1 , -1.53,  1.31,  1.12, -2.05, -4.  , -2.5 , -2.04, -2.33],
       [-2.53,  1.05, -2.69, -2.61,  1.41,  1.77, -2.02, -2.13, -1.46],
       [-1.92, -0.48, -1.42, -1.56, -0.41,  1.55, -1.53,  5.06,  0.39],
       [-1.8 ,  1.21,  0.9 ,  3.17, -1.93,  0.27,  1.08,  3.41,  0.37],
       [-0.58,  1.12,  0.64, -2.18, -0.69, -0.02,  0.31,  2.29, -3.38],
       [ 0.48,  1.47, -3.36,  3.01, -0.27,  3.16,  0.92, -2.53, -1.88],
       [ 2.71,  1.8 ,  4.  , -0.95,  3.86,  1.77, -2.28, -2.13, -0.68],
       [-2.2 , -1.53,  6.06, -2.14,  3.3 , -5.77, -0.45,  1.42,  3.6 ],
       [-2.29,  2.67, -1.62,  0.96,  6.16,  3.12,  3.21, -1.8 , -2.26]])